#

# Fine Tuning Project

#
**Librerias**

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline, T5Tokenizer, T5ForConditionalGeneration, AutoModelForSeq2SeqLM
from torch.utils.data import DataLoader
import shutil
from datasets import load_dataset

#
**Prueba de CUDA**

In [ ]:
device_count = torch.cuda.device_count()
print(f"Number of CUDA devices: {device_count}")

for i in range(device_count):
    print(f"Device {i}: {torch.cuda.get_device_name(i)}")


Number of CUDA devices: 1
Device 0: NVIDIA GeForce RTX 3070 Ti


#
**Dataset**

In [4]:
#Dataset de clasificacion
ds = load_dataset("ccdv/patent-classification", "abstract")

In [5]:
ds_df = pd.DataFrame(ds['train'])
print(ds_df)

                                                    text  label
0      an apparatus for simultaneously testing multip...      6
1      an electrosurgical instrument includes a housi...      0
2      a wireless transceiver device employing the co...      7
3      the invention provides for the use of protein ...      0
4      the present invention relates to a device for ...      8
...                                                  ...    ...
24995  a security tag having a lanyard member that en...      6
24996  in a liquid crystal display device into which ...      6
24997  the present invention provides a process for t...      2
24998  ceiling fan energy consumption efficiency is e...      5
24999  a portable patient supporting seat that is not...      0

[25000 rows x 2 columns]


In [6]:
n_label = ds_df[ds_df["label"] == 8]
n_label

,text,label
4,the present invention relates to a device for ...,8
12,molten salt solar receiver and procedure to re...,8
31,the present invention relates to microencapsul...,8
37,a method of producing nb 3 sn superconducting ...,8
42,a locking device for fastening and / or lockin...,8
...,...,...
24973,the invention relates to concretes and mortars...,8
24981,a fairing removal tool and a method of using t...,8
24982,a plurality of cells installed in a microwave ...,8
24985,electro - magnetic - energy absorbing material...,8


#
**CODIGO COMPLETO**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=9)

# Tokenización del dataset
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Preparar el conjunto de datos para entrenamiento
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]

train_dataloader = DataLoader(train_dataset, batch_size=4, num_workers=4)  
eval_dataloader = DataLoader(eval_dataset, batch_size=4, num_workers=4)

training_args = TrainingArguments(
    output_dir="./classification_model_path",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  
    num_train_epochs=3,
    weight_decay=0.01,  
    logging_dir="./classification_logs",
    logging_steps=10,
    load_best_model_at_end=True,
    disable_tqdm=False, 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=None,  
)

# Entrenar el modelo
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\oswal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\oswal\AppData\Local\Temp\ipykernel_24236\4201625683.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 11/18750 [00:01<28:32, 10.94it/s] 

{'loss': 2.1446, 'grad_norm': 4.934030055999756, 'learning_rate': 1.9989333333333335e-05, 'epoch': 0.0}


  0%|          | 21/18750 [00:02<26:10, 11.93it/s]

{'loss': 2.0931, 'grad_norm': 6.759509563446045, 'learning_rate': 1.997866666666667e-05, 'epoch': 0.0}


  0%|          | 31/18750 [00:02<26:16, 11.87it/s]

...

100%|██████████| 18750/18750 [27:17<00:00, 12.55it/s]

{'eval_loss': 1.2428890466690063, 'eval_runtime': 27.0172, 'eval_samples_per_second': 185.067, 'eval_steps_per_second': 23.133, 'epoch': 3.0}


100%|██████████| 18750/18750 [27:18<00:00, 11.44it/s]

{'train_runtime': 1638.4268, 'train_samples_per_second': 45.776, 'train_steps_per_second': 11.444, 'train_loss': 0.8883448906262715, 'epoch': 3.0}


TrainOutput(global_step=18750, training_loss=0.8883448906262715, metrics={'train_runtime': 1638.4268, 'train_samples_per_second': 45.776, 'train_steps_per_second': 11.444, 'total_flos': 9936295142400000.0, 'train_loss': 0.8883448906262715, 'epoch': 3.0})

In [36]:
model_dir = "./classification_model_path"
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [37]:
tokenizer.save_pretrained("./classification_model_path")

('./classification_model_path\\tokenizer_config.json',
 './classification_model_path\\special_tokens_map.json',
 './classification_model_path\\spiece.model',
 './classification_model_path\\added_tokens.json',
 './classification_model_path\\tokenizer.json')

#
**FINE TUNING CLASSIFICATION**

In [38]:

example_text = "I have to drink water every day"
inputs = tokenizer(example_text, return_tensors="pt", padding=True, truncation=True).to(device)

# Generar predicción
model.eval()  # Configurar el modelo en modo evaluación
with torch.no_grad():
    outputs = model(**inputs)

# Obtener las probabilidades
logits = outputs.logits
predictions = torch.softmax(logits, dim=-1)

# Mostrar los resultados
print(f"Predicciones (probabilidades): {predictions}")
print(f"Etiqueta: {torch.argmax(predictions).item()}")


Predicciones (probabilidades): tensor([[0.1232, 0.1311, 0.0524, 0.0238, 0.0361, 0.0570, 0.3503, 0.1024, 0.1237]],
       device='cuda:0')
Etiqueta: 6


In [ ]:
# Renombrar la ruta para el modelo de resumen
shutil.move('./summarization_model_path', './classification_model_path')


'./classification_model_path'

#
**ZERO SHOT LEARNING CLASSIFICATION**

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

#text = "an apparatus for simultaneously testing multiple integrated circuits includes a sensing circuit associated with each of the tested circuits . each sensing circuit includes a differential amplifier with its positive input connected to the input of the test circuit , and its inversion input connected to the test circuit output . the test circuit input and positive amplifier input are biased to a selected voltage , and the voltage drop across the test circuit is provided to the amplifier inversion input . whenever the test circuit is open , intermittently open or highly resistive , the voltage drop across the test circuit exceeds the threshold voltage of the differential amplifier , causing the amplifier to generate a high level logic output representing an open circuit condition . the outputs of the various sensing circuits together form a digital word representative of the condition of all of the test circuits . the outputs of the differential amplifiers also are provided to independent triggering circuitry for enabling the storage of sensing circuit outputs upon an open condition indicated for at least one of the test circuits . the outputs of the sampling circuits are sampled in parallel at 100 nanosecond or longer selected intervals , so that extremely brief intermittent opens are detected"
text = "I have to drink water every day"

labels = ["Human Necessities", "Performing Operations; Transporting", "Chemistry; Metallurgy",
"Fixed Constructions", "Mechanical Engineering; Lightning; Heating; Weapons; Blasting", "Physiscs", "Electricity", "General tagging of new or cross-sectional technology"]

result = classifier(text, candidate_labels=labels)
print(result)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'sequence': 'I have to drink water every day', 'labels': ['Human Necessities', 'General tagging of new or cross-sectional technology', 'Fixed Constructions', 'Physiscs', 'Performing Operations; Transporting', 'Mechanical Engineering; Lightning; Heating; Weapons; Blasting', 'Electricity', 'Chemistry; Metallurgy'], 'scores': [0.6565184593200684, 0.11612563580274582, 0.1082141175866127, 0.05288640409708023, 0.03639522194862366, 0.011352711357176304, 0.010689978487789631, 0.007817425765097141]}


#

#
**FEW SHOT LEARNING CLASSIFICATION**

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

few_shot_examples = [
    {"text": "an electrosurgical instrument includes a housing , a shaft extending from the housing , and an end effector assembly attached at a distal end of the shaft . a handle assembly is coupled to the housing and includes a movable handle for manipulating the end effector assembly . an outer sleeve is disposed about the shaft and selectively translatable relative thereto . an energizable member is operably coupled to the outer sleeve . a deployment mechanism is provided including a lever rotatably coupled to the housing and positioned proximally of the movable handle and at least one link member coupled between the lever and the outer sleeve . the link member couple to the outer sleeve distally of the movable handle . rotation of the lever translates the outer sleeve distally to move the outer sleeve over the end effector assembly and simultaneously deploy the energizable member distally past the end effector assembly .", "label": "Human Necessities"},
    
    {"text": "a large pore ceramic membrane is used to separate asphaltenes from heavy crude oil . permeate is recycled to the feed for an initial period , during which the pores are deliberately fouled to reduce pore size . this reduction eventually levels off , at which point recycling is terminated and ultrafiltration is continued thereafter at good flux rates with effective asphaltenes removal .", "label": "Performing Operations; Transporting"},
    
    {"text": "a method of manufacturing solvent - soluble charge transporting materials capable of imparting charge - tranporting properties to polysiloxane resins . these materials have the formula a - r 1 sir 2 3 - n q n ! p where a is a tertiary amine and organic group derived from a compound having charge transporting properties with an ionization potential of 4 . 5 - 6 . 2 ev ; r 1 is an alkylene group of 1 - 18 carbon atoms ; r 2 is a monovalent hydrocarbon group or a monovalent halogen - substituted hydrocarbon group of 1 - 15 carbon atoms ; q is a hydrolyzable group such as -- or 3 where r 3 is an alkyl group of 1 - 6 carbon atoms ; n and p are each 1 - 3 . this silicon - type charge transporting material is characterized by aromatic groups , and alkoxysilyl groups bonded via hydrocarbon groups onto the aromatic rings .", "label": "Chemistry; Metallurgy"},
    
    {"text": "a method for erecting a drilling rig and an apparatus therefor having a mast , a base , a floor support , legs arranged between the base and the floor support , a primary lifting ram in engagement with the mast , and a floor support lifting ram , is disclosed . the method involves raising the mast with the primary lifting ram , raising the floor support with the floor support lifting ram to an intermediate height , engaging the primary lifting ram with the floor support and raising the floor support from the intermediate height to full working height with the mast lifting ram .", "label": "Fixed Constructions"},
    
    {"text": "a thermostat apparatus for controlling a heat pump with supplemental resistive heating as typically contained in a packaged terminal air conditioning unit is disclosed which provides improvements in energy management by reducing energy usage as much as possible within user guidelines when the conditioned space served by the ptac is not occupied . a reduced setpoint temperature determination is based upon measurements of thermal response within the conditioned space to application of supplemental or resistive heat , and utilizes supplemental or resistive heat in achieving a quick recovery when people enter the conditioned space , thus enabling a more reduced setpoint temperature when the space is not occupied .", "label": "Mechanical Engineering; Lightning; Heating; Weapons; Blasting"},
    
    {"text": "an apparatus for simultaneously testing multiple integrated circuits includes a sensing circuit associated with each of the tested circuits . each sensing circuit includes a differential amplifier with its positive input connected to the input of the test circuit , and its inversion input connected to the test circuit output . the test circuit input and positive amplifier input are biased to a selected voltage , and the voltage drop across the test circuit is provided to the amplifier inversion input . whenever the test circuit is open , intermittently open or highly resistive , the voltage drop across the test circuit exceeds the threshold voltage of the differential amplifier , causing the amplifier to generate a high level logic output representing an open circuit condition . the outputs of the various sensing circuits together form a digital word representative of the condition of all of the test circuits . the outputs of the differential amplifiers also are provided to independent triggering circuitry for enabling the storage of sensing circuit outputs upon an open condition indicated for at least one of the test circuits . the outputs of the sampling circuits are sampled in parallel at 100 nanosecond or longer selected intervals , so that extremely brief intermittent opens are detected .", "label": "Physiscs"},
    
    {"text": "a wireless transceiver device employing the code select code division multiple access method includes an encoder configured to execute error correction coding on data having a predetermined number of bits among data inputted from outside , and a code selector configured to select a code corresponding to the data subjected to correction coding . the wireless transceiver device performs wireless communication by use of the data inputted from the outside excluding the data having the predetermined number of bits and by use of the code .", "label": "Electricity"},
    
    {"text": "the present invention relates to a device for mixing fluids . it is a hydraulic or pneumatic apparatus , depending on the fluid used for transportation . it is static and has the characteristics of both an extractor and a fluid mixer . extraction is effected by dragging the suction elements , by means of the circulation of a transporting fluid injected at low pressure . the injection inlets and suction inlets are interchangeable and lead to a single outlet . the injection tube formed by a helical spiral on the outside surrounded by the sheath increases the pressure in the transporting fluid and creates outward helical movement with centrifugal force in all the fluid that circulates on the outside .", "label": "General tagging of new or cross-sectional technology"}
]

new_text = "I have to drink water every day"

result = classifier(new_text)

print("Classification result:", result)



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Classification result: [{'label': 'LABEL_0', 'score': 0.5243759751319885}]


#
**SUMMARIZATION**

In [ ]:
ds = load_dataset("stacked-summaries/stacked-samsum-1024")

ds

DatasetDict({
    train: Dataset({
        features: ['dialogue', 'summary', 'chapter_length', 'summary_length', 'is_stacked'],
        num_rows: 29442
    })
    validation: Dataset({
        features: ['dialogue', 'summary', 'chapter_length', 'summary_length', 'is_stacked'],
        num_rows: 1633
    })
    test: Dataset({
        features: ['dialogue', 'summary', 'chapter_length', 'summary_length', 'is_stacked'],
        num_rows: 1637
    })
})

In [ ]:
#Modelo T5-small
model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

def preprocess_data(examples):
    inputs = ["summarize: " + dialogue if dialogue else "summarize: Empty dialogue" for dialogue in examples["dialogue"]]
    targets = examples["summary"]
    
    # Tokenizar las entradas y las salidas
    model_inputs = tokenizer(inputs, max_length=1024, padding="max_length", truncation=True)
    labels = tokenizer(targets, max_length=150, padding="max_length", truncation=True)
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Preprocesar los datos de entrenamiento 
train_dataset = ds["train"].map(preprocess_data, batched=True)
val_dataset = ds["validation"].map(preprocess_data, batched=True)

# Establecer los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  
    save_strategy="epoch",  
    load_best_model_at_end=True,  
    metric_for_best_model="eval_loss", 
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
)
# Crear el trainer
trainer = Trainer(
    model=model,                         
    args=training_args,                 
    train_dataset=train_dataset,     
    eval_dataset=val_dataset,            
)

# Entrenar el modelo
#trainer.train()

# Guardar el modelo entrenado
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")


('./finetuned_model\\tokenizer_config.json',
 './finetuned_model\\special_tokens_map.json',
 './finetuned_model\\spiece.model',
 './finetuned_model\\added_tokens.json')

#
**SUMMARIZATION FINE TUNING**

In [ ]:
# Cargar el modelo entrenado
model_path = "./finetuned_model"  
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)


In [41]:
test_text = """
Logan: So what happens now? Amelia: They vote, I guess? Logan: I ask because of these legal advice things today. It doesn't sound positive. Amelia: Nothing about this sounds positive. Logan: It's the customs union that's the problem right now. Amelia: Seems to be. Logan: No, it is, because they are saying it isn't legal to exit without another customs agreement or something. Amelia: See, how can you be sure what anything says right now? Logan: I'm just reading what the news says... Amelia: Fake news! LOL! Logan: Um, right... Logan: It's the Irish border that's the problem. Ireland is staying in the EU so there has to be a border. Amelia: Ah, and they don't want a hard border? Logan: No. Makes everything more complicated and difficult. Amelia: But this is all lawyer-speak and legalese. Logan: Yes, but all lawyers do is give a frank opinion. Amelia: What does PM say? Logan: She's insistent that there has to be a backstop for NI and the border. Amelia: Well, then... Logan: Some backtracking there, but all in all an honest response from her. Amelia: The whole thing bores me to tears. Logan: More to discuss down the pub! LOL! Amelia: Meet you there!
"""

# Tokenizar el texto
inputs = tokenizer.encode("summarize: " + test_text, return_tensors="pt", max_length=1024, truncation=True)

In [43]:
outputs = model.generate(
    inputs, 
    max_length=150,  
    min_length=30,   
    length_penalty=2.0,  
    num_beams=4,     
    early_stopping=True
)

summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Resumen generado:", summary)


Resumen generado: amelia: "it's the customs union that's the problem right now" amelia: "it's the Irish border that's the problem" amelia: "it's the customs union that's the problem"


#
**FEW SHOT LEARNING - SUMMARIZATION**

In [44]:
from transformers import pipeline

#Modelo de bart-large
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Ejemplos de Few-Shot
few_shot_examples = [
    {"text": "Tinley: How would you spend your vacation? Crew: Maybe I would go to New tork Tinley: Cool Crew: Whats your favorite place youve been? Tinley: I have been to Melbourne Crew: Dats a nice city to visit Tinley: Yeah it is Crew: Should we go together this year? Tinley: I will try to manage my budget Crew: You will have to Tinley: what about the permission from parents? Crew: I will convince them dont worry Tinley: What about yours Crew: It might be difficult for me Tinley: You want me to talk to them Crew: Let it be for now Tinley: oke Crew: Will talk when the time is close Tinley: k sure <3 Sam: r u ready 4 tomorrow exam? Cal: not realy, no. U? Sam: More or less, we'll see Cal: Hope it won't be too difficult like last year... Sam: yeah, that was massacre Alex: We're leaving with Eusebio for 2 weeks, anybody would like to take care of our plants or even stay in the apartment? Eusebio: I will leave you a delicious, fresh apple pie in the fridge! Michael: I can't, sadly Ron: when are you leaving. I can stay in your apartment with my boyfriend, we were planning to visit Vienna anyway Alex: between 15 and 30 of May Eusebio: but you can stay longer even if we're home Ron: Perfect, I'll talk to Marco and let you know Alex: and Vienna is amazing this time of the year, green already and not too hot, you'd have an amazing time, I'm sure Ron: Marco seems positive about the idea. Eusebio: I'm so happy! Ron: What will we have to do? Alex: nothing really apart from watering the plants Ron: Ok, we will be good to them, I promise Alex: I know:) Mum: thank you for a lovely weekend darling Shirley: thats ok mum it was great to see you Mum: have you finished the fireplace wall? Shirley: Not quite.. I need to finish the backing Mum: did you find a big enough piece of cardboard? Shirley: Yes there was some in the shed Mum: how are you going to fix it? Shirley: I have a staple gun Mum: will that work? Shirley: it should do Mum: are you going to wall paper inside the fireplace or paint it grey? Shirley: I'm not sure until I've enclosed it I'll see what will look the best Mum: the grey may look the best then the white fire may stand out more Shirley: yes thats what I'm thinking Mum: send me a picture when you have done it Shirley: <file_photo> Mum: wow darling that looks fab Shirley: thanks mum xx Harry: how's life? Jane: fine, just have tons of work Harry: oh sorry to hear that Jane: can we call eachother up in the evening? Harry: sure when do you get off work Jane: around 6 Harry: ok be in touch Marie: <file_video> Anna: omg he's so cute!! Marie: I know Dorothy: Are you still thinking of ordering a pizza? Ben: Yes. There are 4 people for a pizza. Are you in? I need to order by 12.30. Dorothy: I'm in. I think I'll starve on this green diet.Make sure there is extra cheese :-) Ben: Will do :-) Dorothy: Thx. You're my saviour :-) Let me know when the pizza guy gets here. Ben: Sure. Delphine: i missed you after concert. Zoe: i was in a hurry. How are you? Delphine: will you have time this week for a lunch? Zoe: sorry, i'll be in Paris for 2 weeks. When i'm back with pleasure"},
    
    {"text": "Tinley and Crew want to go to Melbourne together. First they have to convince their parents to let them. [NEXT_CONCEPT] Sam is more prepared for the exam than Cal. They hope that the exam will not be as difficult as last year. [NEXT_CONCEPT] Ron and Marco will move in to Alex' and Eusebio's for 2 weeks to take care of their plants while their away. [NEXT_CONCEPT] Mum and Shirley saw each other in the weekend. Shirley hasn't finished the fireplace wall yet. Shirley needs to finish the backing and cover the inside of the fireplace. Shirley sends Mum a photo, when the work is finished. [NEXT_CONCEPT] Jane and Harry will talk on the phone in the evening. [NEXT_CONCEPT] Marie and Anna think he's cute. [NEXT_CONCEPT] Ben orders a pizza with friends. Dorothy wants to join. He needs to order by 12.30. Dorothy wants extra cheese. [NEXT_CONCEPT] Delphine didn't manage to talk to Zoe after the concert. They will have lunch together after Zoe returns from her 2-week trip to Paris."},
]

# Nuevo texto a resumir
new_text = """
Logan: So what happens now? Amelia: They vote, I guess? Logan: I ask because of these legal advice things today. It doesn't sound positive. Amelia: Nothing about this sounds positive. Logan: It's the customs union that's the problem right now. Amelia: Seems to be. Logan: No, it is, because they are saying it isn't legal to exit without another customs agreement or something. Amelia: See, how can you be sure what anything says right now? Logan: I'm just reading what the news says... Amelia: Fake news! LOL! Logan: Um, right... Logan: It's the Irish border that's the problem. Ireland is staying in the EU so there has to be a border. Amelia: Ah, and they don't want a hard border? Logan: No. Makes everything more complicated and difficult. Amelia: But this is all lawyer-speak and legalese. Logan: Yes, but all lawyers do is give a frank opinion. Amelia: What does PM say? Logan: She's insistent that there has to be a backstop for NI and the border. Amelia: Well, then... Logan: Some backtracking there, but all in all an honest response from her. Amelia: The whole thing bores me to tears. Logan: More to discuss down the pub! LOL! Amelia: Meet you there!
"""

# Generar resumen para el nuevo texto con los ejemplos de Few-Shot
few_shot_summaries = []

for example in few_shot_examples:
    few_shot_summaries.append(summarizer(example['text'])[0]['summary_text'])

# Ahora genera el resumen del nuevo texto
new_summary = summarizer(new_text)[0]['summary_text']

# Mostrar resultados
print("Few-Shot Summaries:")
for example, summary in zip(few_shot_examples, few_shot_summaries):
    print(f"Text: {example['text'][:150]}... => Summary: {summary}")

print("\nSummarization para new_text:")
print(new_summary)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Few-Shot Summaries:
Text: Tinley: How would you spend your vacation? Crew: Maybe I would go to New tork Tinley: Cool Crew: Whats your favorite place youve been? Tinley: I have ... => Summary: Tinley: How would you spend your vacation? Crew: Maybe I would go to New tork Tinley: Cool Crew: Whats your favorite place youve been? Tinley; I have been to Melbourne Crew: Dats a nice city to visit Tinley, Yeah it is Crew: Should we go together this year?
Text: Tinley and Crew want to go to Melbourne together. First they have to convince their parents to let them. [NEXT_CONCEPT] Sam is more prepared for the e... => Summary: Tinley and Crew want to go to Melbourne together. Sam is more prepared for the exam than Cal. Ron and Marco will move in to Alex' and Eusebio's for 2 weeks to take care of their plants while their away. Mum and Shirley saw each other in the weekend.

Summarization para new_text:
Logan: So what happens now? Amelia: They vote, I guess? Logan: I ask because of these legal advice

#
**ZERO SHOT LEARNING - SUMMARIZATION**

In [ ]:

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Texto a resumir
new_text_zero_shot = """
Logan: So what happens now? Amelia: They vote, I guess? Logan: I ask because of these legal advice things today. It doesn't sound positive. Amelia: Nothing about this sounds positive. Logan: It's the customs union that's the problem right now. Amelia: Seems to be. Logan: No, it is, because they are saying it isn't legal to exit without another customs agreement or something. Amelia: See, how can you be sure what anything says right now? Logan: I'm just reading what the news says... Amelia: Fake news! LOL! Logan: Um, right... Logan: It's the Irish border that's the problem. Ireland is staying in the EU so there has to be a border. Amelia: Ah, and they don't want a hard border? Logan: No. Makes everything more complicated and difficult. Amelia: But this is all lawyer-speak and legalese. Logan: Yes, but all lawyers do is give a frank opinion. Amelia: What does PM say? Logan: She's insistent that there has to be a backstop for NI and the border. Amelia: Well, then... Logan: Some backtracking there, but all in all an honest response from her. Amelia: The whole thing bores me to tears. Logan: More to discuss down the pub! LOL! Amelia: Meet you there!
"""

zero_shot_summary = summarizer(new_text_zero_shot)[0]['summary_text']

print("Summarization:", zero_shot_summary)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Summarization: Logan: So what happens now? Amelia: They vote, I guess? Logan: I ask because of these legal advice things today. It doesn't sound positive. Amelia: Nothing about this sounds positive. Logan: It's the customs union that's the problem right now.
